In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

In [2]:
import numpy as np

In [3]:
(X_train,_),(X_test,_) = mnist.load_data()

In [4]:
# Noramlizing between -0.5 and 0.5
X_train = X_train.astype('float32')/255 - 0.5
X_test = X_test.astype('float32')/255 - 0.5

In [5]:
X_train = X_train
X_test = X_test

In [6]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Reshape, Flatten, Conv2D, Conv2DTranspose, MaxPool2D, UpSampling2D

In [7]:
img_shape = X_train[0].shape
img_shape

(28, 28)

In [8]:
X_train = np.reshape(X_train, (len(X_train), 28, 28, 1))
X_test = np.reshape(X_test, (len(X_test), 28, 28, 1))

In [9]:
img_shape = X_train.shape
img_shape

(60000, 28, 28, 1)

In [10]:
input_image = Input(shape=(28,28,1))

In [11]:
#Defining the Convolutional Encoder-Decoder

In [12]:
autoencoder = Sequential()
autoencoder.add(Conv2D(64, kernel_size=3, activation='relu', padding='same', input_shape=(28,28,1)))
autoencoder.add(MaxPool2D((2,2), padding='same'))

autoencoder.add(Conv2D(32, kernel_size=3, activation='relu', padding='same'))
autoencoder.add(MaxPool2D((2,2), padding='same'))

autoencoder.add(Conv2D(10, kernel_size=3, activation='relu', padding='same'))
autoencoder.add(MaxPool2D((2,2), padding='same'))

In [13]:
autoencoder.add(Conv2D(10,kernel_size=3,activation='relu',padding='same'))
autoencoder.add(UpSampling2D((2,2)))

autoencoder.add(Conv2D(32,kernel_size=3,activation='relu',padding='same'))
autoencoder.add(UpSampling2D((2,2)))

autoencoder.add(Conv2D(64,kernel_size=3,activation='relu',padding='same'))
autoencoder.add(UpSampling2D((2,2)))

autoencoder.add(Conv2D(1,kernel_size=3))
autoencoder.add(Conv2D(1,kernel_size=3))
# autoencoder.add(Conv2D(1,kernel_size=3))

In [14]:
for layer in autoencoder.layers:
    print(layer.input_shape)

(None, 28, 28, 1)
(None, 28, 28, 64)
(None, 14, 14, 64)
(None, 14, 14, 32)
(None, 7, 7, 32)
(None, 7, 7, 10)
(None, 4, 4, 10)
(None, 4, 4, 10)
(None, 8, 8, 10)
(None, 8, 8, 32)
(None, 16, 16, 32)
(None, 16, 16, 64)
(None, 32, 32, 64)
(None, 30, 30, 1)


In [15]:
input_ = input_image

In [16]:
# #Using Latent to get the output
# bottleneck = encoder(input_)
# reconstruction = decoder(bottleneck)

In [17]:
#this is the input and output
# autoencoder = Model(input_image,autoencoder)

In [19]:
# print(input_.shape)
# print(reconstruction.shape)

In [21]:
autoencoder.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [22]:
# Lets keep model checkpoints and callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [23]:
checkpoints = ModelCheckpoint(
    filepath='model.h5',
    verbose=1,
    save_best_only=True #Best Model is saved on longer training
)

In [24]:
tb = TensorBoard(
    log_dir="./logs",
    write_graph=True
)

In [25]:
#Since we are reconstructing the same image the X and the Y are the same
history = autoencoder.fit(
    X_train,
    X_train,
    epochs=1,
    batch_size=256,
    shuffle=True,
    validation_data=(X_test,X_test),
    verbose=1,
    callbacks=[checkpoints, tb]
)

Train on 60000 samples, validate on 10000 samples
59904/60000 [============================>.] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 00001: val_loss improved from inf to 0.00000, saving model to model.h5
60000/60000 [==============================] - 222s 4ms/sample - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00


In [26]:
import matplotlib.pyplot as plt

In [222]:
# def show(x):
#     plt.imshow(np.clip(x+0.5,0,1))

In [27]:
# show(X_train[0])

In [29]:
# #expects batch size show like 1*28*28 but we have not give a batch size so, we add none
# autoencoder.predict(X_test[None][0])

In [30]:
# def visualize(img,encoder,decoder):
#     bottleneck = encoder.predict(img[None])[0]
#     print(bottleneck.shape)
#     print(bottleneck[None].shape)
#     reconstruction = decoder.predict(bottleneck[None])[0]
    
#     plt.subplot(1,3,1)
#     show(img)
    
#     plt.subplot(1,3,2)
#     plt.imshow(bottleneck.reshape((bottleneck.shape[-1],1)))
    
#     plt.subplot(1,3,3)
#     plt.imshow(reconstruction)
    
#     plt.show()
    

In [31]:
# for i in range(5):
#     img = X_test[i]
#     visualize(img,encoder,decoder)

In [ ]:
n = 7
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(1, n, i+1)
    plt.imshow(X_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
reconstructed_images = model.predict(X_test)

In [ ]:
n = 7
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i + n + 1)
    plt.imshow(reconstructed_images[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()